In [334]:
import ee
import geemap
ee.Initialize()

In [329]:
import requests
import pandas as pd
import json
import time

In [368]:
GEOCODE = '5107925'
Escala = 350;
url = f'http://servicodados.ibge.gov.br/api/v3/malhas/municipios/{GEOCODE}?formato=application/vnd.geo+json'
response = requests.get(url)
geo = response.json();
ee_object = geemap.geojson_to_ee(geo)
geometry = ee_object.geometry()

In [321]:
response

<Response [200]>

In [338]:
Map = geemap.Map(center=[-11.77,-45.76], zoom=8)

Map.addLayer(geometry, {}, "Geometria")

Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.

Map


Map(center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [339]:
#Se precisar especificar a geometria manualmente no mapa

#Map.draw_features
#roi = ee.FeatureCollection(Map.draw_features)
#geometry = roi.geometry()
#Map

Map(bottom=35429.33331298828, center=[-12.861789351305879, -54.40965085262453], controls=(WidgetControl(option…

In [369]:

def clusterization(image):

          
    # Load a pre-computed Landsat composite for input.
    input = image.clip(geometry).select("EVI");
  
    #Make the training dataset.
    training = input.sample(**{
    'region': geometry,
    'scale': Escala,
    'numPixels': 5000
    });

    #Instantiate the clusterer and train it.
    clusterer = ee.Clusterer.wekaKMeans(5).train(training)
    #Cluster the input using the trained clusterer.
    result = input.cluster(clusterer)
 
    #Combine the mean and standard deviation reducers.
    reducers = ee.Reducer.mean()

    #Inicialização Lista de EVI dos clustes
    lista = ee.List([])

    #Iteração sobre os 5 Clusters para calcular a estatística de cada região
    for i in range(5):

        #Cálculo da geometria
        geom = result.select("cluster").eq(i).selfMask().reduceToVectors(**{
        'scale':Escala,
        });



        #Reducer para cálculo do EVI médio de cada cluster
        stats = image.reduceRegion(**{
        'reducer': reducers,
        'geometry':geom.geometry(),
        'scale':Escala,
        'bestEffort':True,
        'maxPixels':10000000000,
        });

        #Armarzena o EVI médio de cada clúster em uma lista
        lista = lista.add(stats.get("EVI"));


    #Cópia da Lista para ordenar os 2 maiores EVIs
    lista2 = lista;


    #Ordenar a lista para pegar os 2 maiores valores de EVI médio nos clusters
    #Agora nossos clustests de interesse estão nas posições 3 e 4 da lista 2
    lista2 = lista2.sort()

    #Pegar as chaves dos clusteres de interesse na lista original
    chaves1 = lista.indexOf(lista2.get(3))
    chaves2 = lista.indexOf(lista2.get(4))


    #Cálculo da Geometria final

    geom1 = result.select("cluster").eq(chaves1).selfMask().reduceToVectors(**{
    'scale':Escala,
    });

    geom2 = result.select("cluster").eq(chaves2).selfMask().reduceToVectors(**{
    'scale':Escala,
    });

    #Geom é a feature collection de nossos clusters de interesse
    geom = geom1.merge(geom2);



    stats = image.reduceRegion(**{
    'reducer': reducers,
    'geometry':geom.geometry(),
    'scale':Escala,
    'bestEffort':True,
    'maxPixels':10000000000,
    });


    return image.set('EVI_Cluster', stats.get("EVI"))
    



In [496]:
import requests
import pandas as pd
import json
import time
#import os

#out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')


stats = list();

In [498]:
#Dividir o processamento das imagens de acordo com o ano para não sobrecarregar o sistema
ano = list(range(2000, 2022))

#string_ints = [str(int) for int in ano]
#stats = list();
    
for i in (ano):
    
    print ("Safra",i-1,"/",i)
    #Filtrar a territorialidade 
    #4 principais meses de safra
    #filter start and end date   
    mcollection = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(geometry).filterDate( str(i-1) + '-11-01', str(i-1) +'-12-31').filter(ee.Filter.calendarRange(11, 12,'month')).map(clusterization)
    mcollection2 = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(geometry).filterDate( str(i) + '-01-01', str(i) +'-02-28').filter(ee.Filter.calendarRange(1, 2,'month')).map(clusterization)    
    mcollection = mcollection.merge(mcollection2)
    
    #Pegar a propriedade que nos interessa em cada imagem - o EVI_Cluster Calculado 
    a = mcollection.aggregate_array('EVI_Cluster')
    
    
    #10s de soneca pra o GEE não derrubar as requests
    time.sleep(10)
    # Fazer a informação passar do servidor do google para o jupyter
    x = a.getInfo()
    #Salvar cada ano
    stats.append(x)    
    

Safra 1999 / 2000
Safra 2000 / 2001
Safra 2001 / 2002
Safra 2002 / 2003
Safra 2003 / 2004
Safra 2004 / 2005
Safra 2005 / 2006
Safra 2006 / 2007
Safra 2007 / 2008
Safra 2008 / 2009
Safra 2009 / 2010
Safra 2010 / 2011
Safra 2011 / 2012
Safra 2012 / 2013
Safra 2013 / 2014
Safra 2014 / 2015
Safra 2015 / 2016
Safra 2016 / 2017
Safra 2017 / 2018
Safra 2018 / 2019
Safra 2019 / 2020
Safra 2020 / 2021


In [499]:
stats

[[4994.80455371195],
 [4591.214059100032,
  5249.324957050189,
  7261.830082112831,
  7842.22075270021,
  7496.82315846796,
  7020.030394621569,
  6743.350563193284],
 [5098.678410506416,
  4397.764895353538,
  6281.613365855746,
  6100.376751339076,
  6634.259558362253,
  7133.88911905703,
  6218.781076710411,
  5648.500119256094],
 [4990.209681215065,
  5503.902271002472,
  5744.410614805681,
  7493.165250414114,
  7064.040935890057,
  7319.19376210911,
  6813.312297922682,
  5426.8229227498705],
 [5258.901261479877,
  6375.480959667221,
  7290.765226899024,
  6670.028510883837,
  6960.307367937221,
  6482.320287639133,
  5778.640120784228,
  4782.054933236934],
 [5860.137548164352,
  6868.347424456546,
  7406.4006150511805,
  8060.766952288269,
  7011.456974723902,
  6876.674547489325,
  5723.429344139436],
 [5148.7129444427,
  5902.161983148023,
  6489.888935446242,
  7566.889836154395,
  8105.824325477352,
  7563.699662975016,
  5644.685906141845,
  5798.860148694882],
 [5550.1122

In [500]:
df = pd.DataFrame({'lin':stats})

In [501]:
#Separate comma
#df = df.apply(lambda x:pd.Series(x))

In [502]:
df = pd.DataFrame(stats, columns = ['EVI1', 'EVI2','EVI3','EVI4','EVI5','EVI6','EVI7','EVI8'])

In [503]:
df

,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8
0,4994.804554,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4591.214059,5249.324957,7261.830082,7842.220753,7496.823158,7020.030395,6743.350563,NaN
2,5098.678411,4397.764895,6281.613366,6100.376751,6634.259558,7133.889119,6218.781077,5648.500119
3,4990.209681,5503.902271,5744.410615,7493.165250,7064.040936,7319.193762,6813.312298,5426.822923
4,5258.901261,6375.480960,7290.765227,6670.028511,6960.307368,6482.320288,5778.640121,4782.054933
5,5860.137548,6868.347424,7406.400615,8060.766952,7011.456975,6876.674547,5723.429344,NaN
6,5148.712944,5902.161983,6489.888935,7566.889836,8105.824325,7563.699663,5644.685906,5798.860149
7,5550.112270,5643.538460,6589.223150,8140.713381,7757.334177,7227.074059,4758.209708,4955.985075
8,4832.350373,5492.552292,7501.986855,7994.690091,7782.034366,7123.561071,6357.858671,5035.329083
9,6208.318706,7654.436495,7971.274659,8329.527352,7505.875456,6407.588973,5413.511730,NaN


In [377]:
#Reshape da 1a linha - Imagens do MODIS a partir de Mar/2000
#new_order =  ['EVI6', 'EVI7', 'EVI8','EVI1','EVI2','EVI3','EVI4','EVI5'] # specify new order of the third row
#i = 0 # specify row number
#df.loc[0] = df.loc[0, new_order].values

In [504]:
#Corrigir os Nans
#Features dos EVIS pronta
df.fillna(0)


,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8
0,4994.804554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,4591.214059,5249.324957,7261.830082,7842.220753,7496.823158,7020.030395,6743.350563,0.000000
2,5098.678411,4397.764895,6281.613366,6100.376751,6634.259558,7133.889119,6218.781077,5648.500119
3,4990.209681,5503.902271,5744.410615,7493.165250,7064.040936,7319.193762,6813.312298,5426.822923
4,5258.901261,6375.480960,7290.765227,6670.028511,6960.307368,6482.320288,5778.640121,4782.054933
5,5860.137548,6868.347424,7406.400615,8060.766952,7011.456975,6876.674547,5723.429344,0.000000
6,5148.712944,5902.161983,6489.888935,7566.889836,8105.824325,7563.699663,5644.685906,5798.860149
7,5550.112270,5643.538460,6589.223150,8140.713381,7757.334177,7227.074059,4758.209708,4955.985075
8,4832.350373,5492.552292,7501.986855,7994.690091,7782.034366,7123.561071,6357.858671,5035.329083
9,6208.318706,7654.436495,7971.274659,8329.527352,7505.875456,6407.588973,5413.511730,0.000000


In [505]:
url = f'https://apiprevmet3.inmet.gov.br/estacao/proxima/{GEOCODE}'
response = requests.get(url).json()
response

{'error': 'Nenhuma estação próxima encontrada'}

In [506]:
estacao = response['dados']['CD_ESTACAO']

KeyError: 'dados'

In [ ]:
new_url = f'https://apitempo.inmet.gov.br/estacao/diaria/1999-11-01/2021-04-30/{estacao}'
json_response = requests.get(new_url).json()
dados_estacao = pd.DataFrame(json_response)
dados_estacao

In [ ]:
dados_estacao['CHUVA'] = dados_estacao['CHUVA'].astype(float)
dados_estacao['DT_MEDICAO'] = pd.to_datetime(dados_estacao['DT_MEDICAO'])
dados_estacao['ANO'] = dados_estacao['DT_MEDICAO'].dt.year
dados_estacao['MES'] = dados_estacao['DT_MEDICAO'].dt.month
dados_estacao

In [ ]:
b  = dados_estacao.groupby(['ANO', 'MES'])['CHUVA'].sum().fillna(0)

In [507]:
mask = (dados_estacao['MES'] <= 4) | (dados_estacao['MES'] >= 11)  
b = dados_estacao.loc[mask].groupby(['ANO', 'MES'])['CHUVA'].sum().fillna(0)

In [508]:
chuva = b.values.reshape(22,6)
df2 = pd.DataFrame(chuva, columns = ['C_NOV', 'C_DEZ','C_JAN','C_FEV','C_MAR','C_ABR'])
df2

,C_NOV,C_DEZ,C_JAN,C_FEV,C_MAR,C_ABR
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0


In [509]:
concat_data = pd.concat([df,df2],axis=1).fillna(0)

In [510]:
concat_data

,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8,C_NOV,C_DEZ,C_JAN,C_FEV,C_MAR,C_ABR
0,4994.804554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,4591.214059,5249.324957,7261.830082,7842.220753,7496.823158,7020.030395,6743.350563,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,5098.678411,4397.764895,6281.613366,6100.376751,6634.259558,7133.889119,6218.781077,5648.500119,0.0,0.0,0.0,0.0,0.0,0.0
3,4990.209681,5503.902271,5744.410615,7493.165250,7064.040936,7319.193762,6813.312298,5426.822923,0.0,0.0,0.0,0.0,0.0,0.0
4,5258.901261,6375.480960,7290.765227,6670.028511,6960.307368,6482.320288,5778.640121,4782.054933,0.0,0.0,0.0,0.0,0.0,0.0
5,5860.137548,6868.347424,7406.400615,8060.766952,7011.456975,6876.674547,5723.429344,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
6,5148.712944,5902.161983,6489.888935,7566.889836,8105.824325,7563.699663,5644.685906,5798.860149,0.0,0.0,0.0,0.0,0.0,0.0
7,5550.112270,5643.538460,6589.223150,8140.713381,7757.334177,7227.074059,4758.209708,4955.985075,0.0,0.0,0.0,0.0,0.0,0.0
8,4832.350373,5492.552292,7501.986855,7994.690091,7782.034366,7123.561071,6357.858671,5035.329083,0.0,0.0,0.0,0.0,0.0,0.0
9,6208.318706,7654.436495,7971.274659,8329.527352,7505.875456,6407.588973,5413.511730,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [511]:
# Api PAM SIDRA/ https://apisidra.ibge.gov.br/values/t/5457/n6/2919553/v/112/p/last%2020/c782/40124

In [512]:
url3 = f'https://apisidra.ibge.gov.br/values/t/5457/n6/{GEOCODE}/v/112/p/last%2020/c782/40124'
json_response = requests.get(url3).json()
produtividade = pd.DataFrame(json_response)
Y = produtividade['V'].iloc[1:].values.ravel()
Y

array(['3300', '3255', '3120', '3060', '3121', '3120', '3000', '3062',
       '3120', '3120', '2984', '3480', '3239', '3124', '3146', '3148',
       '2880', '3480', '3720', '3540'], dtype=object)

In [513]:
#Produtividade de Soja por ano - PAM/SIDRA
df3 = pd.DataFrame(Y)
df3

,0
0,3300
1,3255
2,3120
3,3060
4,3121
5,3120
6,3000
7,3062
8,3120
9,3120


In [389]:
import sklearn
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [591]:
#X_treino com dados do passado e X_Test com dados do futuro
#Selecionei só 3 medidas do EVI para não ficar muita feature
Xtrain = df.iloc[1:19,1:7].fillna(0)
Xtest =  df.iloc[19:22,1:7]
Ytrain = Y[1:19]
Ytest = Y[19:20]

In [592]:
#Normalizar o Xtrain
#from sklearn import preprocessing
#from sklearn.preprocessing import StandardScaler
#min_max_scaler = preprocessing.MinMaxScaler()
#X_train_minmax = min_max_scaler.fit_transform(Xtrain)
#Xtrain

In [593]:
# Regressão Linear Simples
LR = LinearRegression()
# fitting the training data
LR.fit(Xtrain,Ytrain)
Y_LR =  LR.predict(Xtrain)

In [594]:
#Regressão Polinomial SVR

svr_poly = SVR(kernel='poly', C=1e3, degree=3)
Y_Poly = svr_poly.fit(Xtrain, Ytrain).predict(Xtrain)

In [595]:
#Regressão Ridge
reg = linear_model.Ridge(alpha=.5)
Y_Ridge = reg.fit(Xtrain, Ytrain).predict(Xtrain)

In [596]:
#Regressão Bayseana
regB = linear_model.BayesianRidge()
Y_Bay = regB.fit(Xtrain, Ytrain).predict(Xtrain)

In [597]:
# predicting the accuracy score
score=r2_score(Ytrain,Y_LR)
print("r2 Linear ",score)
print("mean_sqrd_error Linear ==",mean_squared_error(Ytrain,Y_LR))
score=r2_score(Ytrain,Y_Poly)
print("r2 Poly ",score)
print("mean_sqrd_error Polinomial ==",mean_squared_error(Ytrain,Y_Poly))
score=r2_score(Ytrain,Y_Ridge)
print("r2 Ridge ",score)
print("mean_sqrd_error Ridge ==",mean_squared_error(Ytrain,Y_Ridge))
score=r2_score(Ytrain,Y_Bay)
print("r2 Bayseano ",score)
print("mean_sqrd_error Bayseano ==",mean_squared_error(Ytrain,Y_Bay))


r2 Linear  0.4168870163142312
mean_sqrd_error Linear == 22444.54606337456
r2 Poly  0.804960186991446
mean_sqrd_error Polinomial == 7507.258781295577
r2 Ridge  0.41688701631421854
mean_sqrd_error Ridge == 22444.546063375048
r2 Bayseano  0.12060750429187361
mean_sqrd_error Bayseano == 33848.61241289604


In [598]:
print("Produtividade em 2019:",Ytest)
print("Produtividade Reg Linear:",LR.predict(Xtest))
print("Produtividade Reg Polinomial:",svr_poly.predict(Xtest))
print("Produtividade Reg Ridge:",reg.predict(Xtest))
print("Produtividade Reg Bayeseana:",regB.predict(Xtest))

Produtividade em 2019: ['3540']
Produtividade Reg Linear: [3155.52902663 3364.96148207 3234.0968478 ]
Produtividade Reg Polinomial: [3622.07423458 4310.56543503 3688.13665337]
Produtividade Reg Ridge: [3155.52903089 3364.96143216 3234.09683069]
Produtividade Reg Bayeseana: [3201.77813732 3218.71376008 3205.21615568]


In [599]:
#Tentativa de Plotar 1 dimensão de feature x Y
#import matplotlib.pyplot as plt
#import numpy as np

#plt.plot(x_plot,y_plot, 'o')
#plt.plot(x_plot,y_poly)
#plt.plot(x_plot,y_prediction)
#plt.show()

Xtrain

,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7
1,5249.324957,7261.830082,7842.220753,7496.823158,7020.030395,6743.350563
2,4397.764895,6281.613366,6100.376751,6634.259558,7133.889119,6218.781077
3,5503.902271,5744.410615,7493.165250,7064.040936,7319.193762,6813.312298
4,6375.480960,7290.765227,6670.028511,6960.307368,6482.320288,5778.640121
5,6868.347424,7406.400615,8060.766952,7011.456975,6876.674547,5723.429344
6,5902.161983,6489.888935,7566.889836,8105.824325,7563.699663,5644.685906
7,5643.538460,6589.223150,8140.713381,7757.334177,7227.074059,4758.209708
8,5492.552292,7501.986855,7994.690091,7782.034366,7123.561071,6357.858671
9,7654.436495,7971.274659,8329.527352,7505.875456,6407.588973,5413.511730
10,6922.089853,6862.431039,7392.215639,7751.599660,6965.553158,5662.784617
